In [1]:
#copied from traffic_demo.py
%matplotlib inline


In [2]:
#pid,tz = 23618402, 'Europe/Berlin'

pid,tz = 23602975, 'Europe/Amsterdam'

#pid,tz = 20095971,'America/Chicago'
#pid,tz = 20095972,'America/Chicago'

In [3]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]


In [4]:
import os,sys
from hmctools.segment import TopologySegment,HmcPartition
import pytz,datetime

import numpy as np
import pandas as pd

In [5]:
def get_traffic_speeds(pid,sid,orient,offset ,tzstring):

    segment = TopologySegment(pid,sid)

    live_traffic_data = segment.get_live_traffic_speed(offset,orient)
    if live_traffic_data is None:
        return None,None
    live_speed = live_traffic_data[2]
    live_time = live_traffic_data[0]
    if live_traffic_data[1]!=0:
        print('Careful - there is a nonzero value for update time!',live_traffic_data[1])

    #need to convert UTC timestamp to local datetime
    live_utc = datetime.datetime.fromtimestamp(live_time, pytz.timezone('UTC') )
    local_tz = pytz.timezone(tzstring) # e.g. 'Europe/Berlin'
    local_time = live_utc.astimezone(local_tz)

    print("Local time ",local_time)
    pattern_speed = segment.get_historical_traffic_speed(offset,orient, local_time )
    print("Historical/live speed:",pattern_speed,live_speed)

    return pattern_speed,live_speed


In [5]:
def main(pid,tzstring):

    historical_speeds = []
    live_speeds = []
    segids = []
    dots = []
    seglens= []
    segfcs = []
    #just use offset=0.5 of each segment
    offset = 0.5

    partition = HmcPartition(pid)
    all_segment_ids = partition.get_all_segment_ids()
     
    nexcept = 0
    nok = 0
        
    for iseg,seg in enumerate(all_segment_ids):
        segment = TopologySegment(pid,seg)
        aa = segment.accessible_by(offset)
        dot = segment.travel_direction(offset)
        if not aa:
            continue
        if dot=='BOTH':
            dot = ['FORWARD','BACKWARD']
        else:
            dot = [dot]
        segment_length = segment.length()
        fcstring = segment.functional_class(offset)
        if fcstring is not None:
            fc = int( fcstring[-1] )
        else:
            fc =0
        for dd in dot:

            try:
                hist,live = get_traffic_speeds( pid,seg,dd,offset,tzstring )
                historical_speeds.append(hist)
                live_speeds.append(live)
                segids.append(seg)
                dots.append(dd)
                seglens.append(segment_length)
                segfcs.append(fc)
                
                nok += 1
            except:
                nexcept +=1
                #print('warning: exception caught')
        #if iseg>10:
        #    break
    return pd.DataFrame( {'segment':segids,'direction':dots,'historical':historical_speeds,'live':live_speeds,'FC':segfcs,'length':seglens}  )



In [6]:

df = main(pid, tz)
print(df)

NameError: name 'main' is not defined

In [ ]:
df.dropna(subset=['historical','live','FC','length'],inplace=True)

df['residual']=df['live']-df['historical']
df['km']=df['length']/1000.0
df['weighted_residual'] = df['residual']*df['km']
print( df['FC'].unique() )
maxval = max( df['residual'].max(), abs(df['residual'].min())  )
colmap = {1:'k',2:'r',3:'g',4:'b',5:'m'}
binning = np.linspace(-maxval,maxval,26)


In [ ]:
groups = df.groupby('FC')
for ifc, gdata in groups:
    if ifc==5:
        continue
    plt.hist( gdata['residual'],bins=binning,color=colmap[ifc],histtype='step',label='FC'+str(ifc),weights=gdata['km'])
    #ax[1].hist( gdata['residual'],bins=binning,color=colmap[ifc],histtype='step',label='FC'+str(ifc),normed=True)
    print("FC"+str(ifc)+': length-weighted mean = '+str( gdata['weighted_residual'].sum() /gdata['km'].sum() ))

#for i in range(2):
plt.legend()
plt.xlabel('live - historical (kph)')
plt.ylabel('Total km')


In [9]:
df.head()

,segment,direction,historical,live,FC,length,residual,km,weighted_residual
4,76333167,BACKWARD,19.0,22.21,3,24.51817,3.21,0.024518,0.078703
46,76689661,FORWARD,41.0,30.04,2,29.05034,-10.96,0.029050,-0.318392
65,76780856,FORWARD,32.0,29.79,4,39.12890,-2.21,0.039129,-0.086475
87,76951990,FORWARD,42.0,42.65,4,27.27011,0.65,0.027270,0.017726
112,77091139,BACKWARD,36.0,26.40,4,34.50149,-9.60,0.034501,-0.331214


In [ ]:
from nagini.utils.tiling import bounding_box_from_tile_id
clat = bounding_box_from_tile_id(pid).center().latitude
clon = bounding_box_from_tile_id(pid).center().longitude

In [ ]:
cmap_name = 'RdYlGn'#'seismic'
cm = matplotlib.cm.get_cmap(cmap_name)
max_abs_res = max( np.abs( df['residual'] ))


In [ ]:
def get_color(val,max_abs_residual):
    norm = matplotlib.colors.Normalize(vmin=-max_abs_residual, vmax=max_abs_residual)
    ctuple = cm(norm(val),bytes=True)
    cstr = '#'
    for i in range(3):
        cstr+= hex(ctuple[i])[2:].zfill(2)
    return cstr
print(max_abs_res)

In [ ]:
norm=matplotlib.colors.Normalize(vmin=-max_abs_res, vmax=max_abs_res)
gradient = np.linspace(-max_abs_res, max_abs_res, 256)
gradient = np.vstack((gradient, gradient))
plt.imshow(norm(gradient), aspect='auto', cmap=cm)

In [ ]:
import nagini
from ipyleaflet import Map, Polyline
hls = nagini.resource('hls')

mymap = hls.map(center=[clat, clon], zoom=12)

for idx,row in df.iterrows():
    sid = row['segment']
    geom = [ list(q) for q in TopologySegment(pid,sid).shape_points() ]
    #print(sid,geom)
    c = get_color(row['residual'],max_abs_res)
    pl = Polyline(locations = geom,color=c,fill=False)
    mymap.add_layer(pl)


In [ ]:
from ipyleaflet import FullScreenControl
mymap.add_control(FullScreenControl())
mymap